### load model ###

In [8]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn as nn

In [ ]:
model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).to("cpu")



## telechargement local du model LLM ##

In [2]:
# Sauvegarde locale dans un dossier à toi
local_dir = "./phi2_local"
model.save_pretrained(local_dir)
tokenizer.save_pretrained(local_dir)


NameError: name 'model' is not defined

In [2]:
local_dir = "./phi2_local"  # ou chemin absolu
model = AutoModelForCausalLM.from_pretrained(local_dir)
tokenizer = AutoTokenizer.from_pretrained(local_dir)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch

# Chargement du dataset
dataset = load_dataset("json", data_files={"train": "C:/Users/hugom/OneDrive/Documents/Stage_2025/LLM_dataset.jsonl"}, split="train")

# Prétraitement
def format_example(example):
    prompt = f"{example['instruction']}\n\nInput:\n{example['input']}\n\nAnswer:\n"
    full_text = prompt + example["output"]
    return {"text": full_text}

dataset = dataset.map(format_example)
local_dir = "./phi2_local"
model = AutoModelForCausalLM.from_pretrained(local_dir)
tokenizer = AutoTokenizer.from_pretrained(local_dir)
tokenizer.pad_token = tokenizer.eos_token
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512,)

tokenized_dataset = dataset.map(tokenize, batched=True,)

# Configuration d'entraînement
# Arguments d'entraînement simples
training_args = TrainingArguments(
    output_dir="./fine_tuned_llm",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=100,
    save_total_limit=1,
    logging_steps=10
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Lancement du fine-tuning
trainer.train()
trainer.save_model("./fine_tuned_phi")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

C:\Users\hugom\AppData\Local\Temp\ipykernel_10184\2034161181.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\Users\hugom\OneDrive\Documents\Stage_2025\Stage_env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [9]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim=283, hidden_dims=[128, 64], num_classes=12):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(),
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU(),
            nn.Linear(hidden_dims[1], num_classes)
        )

    def forward(self, x):
        return self.model(x)


In [10]:
import torch
def prediction(path_model,x,treshold):
    model = MLPClassifier()
    model.load_state_dict(torch.load(path_model))
    model.eval()
    with torch.no_grad():
        input = x.unsqueeze(0)
        y = model(input)
        y_probs = torch.sigmoid(y)
        y_preds = (y_probs > treshold).int()
    return y_probs,y_preds

In [32]:
import joblib
import json
def predict(X,clf):
    input = X
    input = input.reshape(1, -1)
    y_pred = clf.predict(input)
    return y_pred

clf = joblib.load("C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/modele_multioutput.pkl")
#import data
test = torch.load('C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/temp.pt',weights_only=False)
test_past = torch.load('C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/temp_past.pt',weights_only=False)
test_past_past = torch.load('C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/temp_past_past.pt',weights_only=False)
#import fichier json
with open("C:/Users\hugom\OneDrive\Documents\Stage_2025\data/trends.json", "r") as f:
    data_loaded = json.load(f)
trends = {'occupation_list':data_loaded['occupation_list'],
          'water consumption':data_loaded['water consumption'],
          'electricity consumption':data_loaded['electricity consumption']}
dj = data_loaded["dj"]
temp = data_loaded["temp_ext"]
data = np.array(data_loaded["commandes"])
data = list(data.reshape(-1))
#prediction creation du segment actuellement hybride pcq on a pas les conso mais il manque que ça
X = test['input']
X = X.reshape(-1)
X = X.numpy()
conso = X[240]
X = X[261:]
data.append(conso)
for i in temp:
    data.append(i)
for i in X:
    data.append(i)
X = np.array(data)

y_pred = predict(X,clf)
print(y_pred)

[[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [45]:


prompt = f"""
You are an assistant for hotel energy management.

Given:
- Overheated zones
- Overventilated zones
- Detected anomalies

Your task:

1. List all overheated zones.
2. List all overventilated zones.
3. List all anomalies or leaks that should be checked.
4. Explain why these anomalies could be a problem

Respond strictly with plain text (no Markdown):


Input:
Overheated: room 101, meeting room
Overventilated: room 102
Anomalies: heater on second floor, light on second floor, ventilation disfunction on third floor

Answer:


"""
print(prompt)



You are an assistant for hotel energy management.

Given:
- Overheated zones
- Overventilated zones
- Detected anomalies

Your task:

1. List all overheated zones.
2. List all overventilated zones.
3. List all anomalies or leaks that should be checked.
4. Explain why these anomalies could be a problem

Respond strictly with plain text (no Markdown):


Input:
Overheated: room 101, meeting room
Overventilated: room 102
Anomalies: heater on second floor, light on second floor, ventilation disfunction on third floor

Answer:





In [46]:


inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
outputs = model.generate(**inputs,
                         do_sample=False,
                         max_new_tokens=400,
                         eos_token_id=tokenizer.eos_token_id,
                         pad_token_id=tokenizer.eos_token_id)



In [47]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(len(prompt))
import json
with open("C:/Users\hugom\OneDrive\Documents\Stage_2025\data/assistant.json", "w+") as f:
    json.dump(response, f, indent=4)
    f.close()
with open("C:/Users\hugom\OneDrive\Documents\Stage_2025\data/assistant.json", "r") as f:
    data_loaded = json.load(f)
    print(data_loaded[:])

530

You are an assistant for hotel energy management.

Given:
- Overheated zones
- Overventilated zones
- Detected anomalies

Your task:

1. List all overheated zones.
2. List all overventilated zones.
3. List all anomalies or leaks that should be checked.
4. Explain why these anomalies could be a problem

Respond strictly with plain text (no Markdown):


Input:
Overheated: room 101, meeting room
Overventilated: room 102
Anomalies: heater on second floor, light on second floor, ventilation disfunction on third floor

Answer:


1. Overheated zones: room 101, meeting room
2. Overventilated zones: room 102
3. Anomalies: heater on second floor, light on second floor, ventilation disfunction on third floor
4. These anomalies could be a problem because they indicate that the HVAC system is not functioning properly. The heater on the second floor could be consuming more energy than necessary, leading to higher energy bills. The light on the second floor could be a sign of a faulty sensor, wh